In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import re

import os
from pathlib import Path

from glob import glob
from tqdm import tqdm

import yaml
from yaml import dump
import uuid
import itertools

In [3]:
import warnings
warnings.simplefilter(action="ignore")

In [4]:
def get_paths(models_list):
    
    """
    Finds all the paths to forecasts and experiments metadata (directories /forecast/ and /wf_result/)

    Returns:
        forecast_paths   : list[str]  - шляхи до всіх CSV з прогнозами (останній research_task_* для кожної моделі)
        metadata_paths   : list[str]  - шляхи до всіх CSV з метаданими (останній research_task_* для кожної моделі)
        metadata         : dict       - словник, зчитаний з відповідних YAML файлів (за іменами CSV -> .yaml)
        experiment_names : list[str]  - [f"{model_name}_{short_uuid}"]
    """
    
    
    base_forecast = Path("/masters_diploma/forecast")
    base_info     = Path("/masters_diploma/wf_result")

    forecast_paths = []
    metadata_paths = []
    metadata = {}
    experiment_names = []

    def latest_task_dir(root: Path) -> Path | None:
        candidates = list(root.glob("research_task_*"))
        if not candidates:
            return None
        return max(candidates, key=lambda p: p.stat().st_ctime)

    for model in models_list:
        info_root = base_info / model
        info_task = latest_task_dir(info_root)
        if info_task and info_task.is_dir():
            mp = sorted([str(p) for p in info_task.glob("*.csv")])
            metadata_paths.extend(mp)
            
        pred_root = base_forecast / model
        pred_task = latest_task_dir(pred_root)
        if pred_task and pred_task.is_dir():
            fps = sorted([str(p) for p in pred_task.rglob("*.csv")])
            forecast_paths.extend(fps)

            
    yaml_file_paths = []
    for csv_path in metadata_paths:
        y = Path(csv_path).with_suffix(".yaml")
        if y.exists():
            yaml_file_paths.append(str(y))

    for file in yaml_file_paths:
        try:
            with open(file, "r", encoding="utf-8") as f:
                res = yaml.safe_load(f)
        except Exception:
            continue

        uid = res.get("unique_uuid", "")
        parts = uid.split("-")
        if len(parts) >= 2:
            shorten_uuid = "-".join([parts[0], parts[-2]])
        else:
            shorten_uuid = uid or "unknown"

        dur = res.get("duration_training_history", res.get("train_start"))

        metadata[shorten_uuid] = {
            "uuid": uid,
            "model_name": res.get("model_name"),
            "duration/train_start": dur,
            "hyperparameters": res.get("model_hyperparameters"),
            "features": res.get("train_features"),
        }

        experiment_names.append(f"{res.get('model_name')}_{shorten_uuid}")

        
    return forecast_paths, metadata_paths, metadata, experiment_names

In [5]:
def facts(path_to_all):

    dateparse = lambda dates: datetime.strptime(dates, '%Y-%m-%d %H:%M:%S')
    path_to_weather = f'{path_to_all}/processed_data/history_weather.csv'

    fact_temperature = pd.read_csv(
        path_to_weather,
        parse_dates=['date'],
        index_col='date', 
        date_parser=dateparse
    )[['temperature']]
    
    fact_temperature.index.name = 'date_time'

    return fact_temperature

In [6]:
def make_forecasts_df(fact_pred, paths_to_exp_forecasts, exp_name):

    '''
    Creating a dataframe of forecasted temperature values
    '''

    dateparse = lambda dates: datetime.strptime(dates, '%Y-%m-%d %H:%M:%S')
    
    df = fact_pred.copy()
    
    for num_exp, day_pred in enumerate([paths_to_exp_forecasts]):
        _ = day_pred.split('_')
        if "_".join([_[-4], _[-3]]) == 'random_forest':
            d = _[-5]
        else:
            d = _[-4]
            
        day_date = day_pred.split('\\')[-1].split('_')[-1].split(')')[0].split('(')[1]
#         print(day_date)

        pred = pd.read_csv(
            day_pred,
            parse_dates=['date_time'],
            index_col='date_time', 
            date_parser=dateparse
        )
        
        for h in range(24):
            try:

                df.loc[pd.to_datetime(day_date) + timedelta(hours=h), f'{exp_name}_{d}'] = pred.loc[pd.to_datetime(day_date) + timedelta(hours=h),'0']
            
            except KeyError as e:
                
                print(day_pred)
                continue
                

    return df

In [7]:
def get_stat(fact_pred, info, day, path_to_files):
    
    forecast_cols = [col for col in fact_pred.columns if day in col]
    df = fact_pred[['temperature'] + forecast_cols].dropna()
    
    df.columns = df.columns.str.replace(r'_d-\d+$', '', regex=True)

    
    absolute_errors = df[df.columns[1:]].sub(df['temperature'], axis=0)
    
    relative_errors = absolute_errors.div(df['temperature'], axis=0)
    
#     print(absolute_errors.columns)
    
    stat_dict = {}
    for exp in info.keys():
        exp_name = f"{info[exp]['model_name']}_{exp}"
        
        stat_dict[exp] = {
            'exp_id': exp_name,
            'mean_abs_value': absolute_errors[exp_name].abs().mean(),
            'mean_rel_value': relative_errors[exp_name].abs().mean(),
            'median_abs_value': absolute_errors[exp_name].abs().median(),
            'median_rel_value': relative_errors[exp_name].abs().median(),
            'q25_abs_value': absolute_errors[exp_name].abs().quantile(0.25),
            'q25_rel_value': relative_errors[exp_name].abs().quantile(0.25),
            'q75_abs_value': absolute_errors[exp_name].abs().quantile(0.75),
            'q75_rel_value': relative_errors[exp_name].abs().quantile(0.75),
            "model_name": info[exp]["model_name"],
            "hyperparameters": f'{info[exp]["hyperparameters"]}',
            "features": f'{info[exp]["features"]}',
            "train_start": info[exp]["duration/train_start"]
        }
        
    stat = pd.DataFrame(stat_dict).T
    
    print(stat)
        
    stat_per_h = pd.DataFrame(relative_errors.abs().groupby(df.index.hour).median(), columns=relative_errors.columns)
    
    
    path = os.path.join(path_to_files, 'statistics', f'general_statistics_{day}.xlsx')
    path_h = os.path.join(path_to_files, 'statistics', f'general_statistics_{day}_by_hour.xlsx')
    
    if os.path.exists(path):
    
        gen_stat_df = pd.read_excel(path)
        gen_stat_h_df = pd.read_excel(path_h)
        gen_stat_df = pd.concat([gen_stat_df, stat], axis=0).drop_duplicates()
        gen_stat_h_df = pd.concat([gen_stat_h_df, stat_per_h], axis=0).drop_duplicates()

        gen_stat_df.to_excel(path, index=False)
        gen_stat_h_df.to_excel(path_h, index=False)
        
    else:
        stat.to_excel(path, index=False)
        stat_per_h.to_excel(path_h)
    
    return stat, stat_per_h

In [8]:
def get_best_models_per_hour(stat_per_h, day, info, path_to_files):
    
    min_errors = stat_per_h.min(axis=1)
    best_exps = stat_per_h.idxmin(axis=1)
    
    
    best_models = pd.concat([best_exps, min_errors], axis=1)
    best_models.columns=['experiment_name', 'median_rel_err_value']
    
    for h in best_models.index:
        exp = best_models.loc[h, 'experiment_name']
        meta = info[exp.split('_')[-1]]
        
        best_models.loc[best_models['experiment_name']==exp, 'model_name'] = meta['model_name']
        best_models.loc[best_models['experiment_name']==exp, 'hyperparameters'] = f"{meta['hyperparameters']}"
        best_models.loc[best_models['experiment_name']==exp, 'features'] = f"{meta['features']}"
        best_models.loc[best_models['experiment_name']==exp, 'duration/train_start'] = meta['duration/train_start']
        
        
    path = get_next_versioned_filename(os.path.join(path_to_files, 'statistics', 'best_for_hour'), day)
    best_models.to_excel(path)
    
    
    return best_models

In [9]:
def get_next_versioned_filename(base_dir, day, prefix="hourly_best", ext=".xlsx"):
    today = datetime.today().strftime("%Y-%m-%d")
    pattern = re.compile(rf"{prefix}_{day}_{today}_v(\d+){re.escape(ext)}")
    
    # Отримаємо всі файли в директорії, які відповідають шаблону
    existing_versions = []
    for filename in os.listdir(base_dir):
        match = pattern.match(filename)
        if match:
            existing_versions.append(int(match.group(1)))
    
    next_version = max(existing_versions, default=0) + 1
    new_filename = f"{prefix}_{day}_{today}_v{next_version}{ext}"
    
    
    return os.path.join(base_dir, new_filename)

In [10]:
path_to_all = '/masters_diploma/'
models_list = ['xgboost', 'random_forest', 'lightgbm']

print('gathering experiment info...')
paths, metadata_paths, metadata_dict, exp_names = get_paths(models_list)

print('loading fact temperature dataset...')
fact_temperature = facts(path_to_all)
fact_pred = fact_temperature.copy()

print('adding experiments` forecasts...')


for exp_forecasts in tqdm(paths):
      
    k = exp_forecasts.split("\\")[-2].split('-')
#     k = exp_forecasts[0].split("\\")[-2].split('-')
    exp = "-".join([k[0], k[-2]])
    
    fact_pred = make_forecasts_df(fact_pred, exp_forecasts, exp)
            
            
# for key, metadata in tqdm(metadata_dict.items()):
# #     print(key, metadata)
    
#     exp_name = f"{metadata['model_name']}_{key}"
    
#     for exp_forecasts in paths:
        
#         k = exp_forecasts[0].split("\\")[-2].split('-')
#         exp = "-".join([k[0], k[-2]])
        
#         if exp == exp_name:

#             fact_pred = make_forecasts_df(fact_pred, exp_forecasts, exp)
# #             print(len(fact_pred.columns))
#         else:
#             continue




fact_pred = fact_pred.loc['2025-09-01':'2025-09-14'].dropna()

for d in range(4):
    print(f'\ncalculating statistics for day {d}...')
    stat, stat_per_h = get_stat(fact_pred, metadata_dict, f'd-{d}', path_to_all)
    print('finished')
    print('finding best model for hour...')
    best_models_df = get_best_models_per_hour(stat_per_h, f'd-{d}', metadata_dict, path_to_all)
    print('done\n')

gathering experiment info...
loading fact temperature dataset...
adding experiments` forecasts...


100%|██████████████████████████████████████████████████████████████████████████| 18450/18450 [1:41:11<00:00,  3.04it/s]



calculating statistics for day 0...
                               exp_id mean_abs_value mean_rel_value  \
4332760e-9d95   xgboost_4332760e-9d95        1.81055       0.094177   
4332760f-a343   xgboost_4332760f-a343       3.391371          0.171   
43327610-b28f   xgboost_43327610-b28f       2.678605       0.139077   
43327611-b4c2   xgboost_43327611-b4c2       1.873648       0.097301   
43327612-8f45   xgboost_43327612-8f45       3.511733       0.177565   
...                               ...            ...            ...   
4332773e-aa60  lightgbm_4332773e-aa60      17.582625        0.94453   
4332773f-9505  lightgbm_4332773f-9505      17.582798       0.944541   
43327740-8500  lightgbm_43327740-8500      17.581506       0.944469   
43327741-b825  lightgbm_43327741-b825      17.581499       0.944468   
43327742-9534  lightgbm_43327742-9534      17.581569       0.944473   

              median_abs_value median_rel_value q25_abs_value q25_rel_value  \
4332760e-9d95         1.508886 

finished
finding best model for hour...
done


calculating statistics for day 2...
                               exp_id mean_abs_value mean_rel_value  \
4332760e-9d95   xgboost_4332760e-9d95        1.81055       0.094177   
4332760f-a343   xgboost_4332760f-a343       3.331597       0.167071   
43327610-b28f   xgboost_43327610-b28f       2.678605       0.139077   
43327611-b4c2   xgboost_43327611-b4c2       1.873648       0.097301   
43327612-8f45   xgboost_43327612-8f45       3.399608       0.171265   
...                               ...            ...            ...   
4332773e-aa60  lightgbm_4332773e-aa60      17.582625        0.94453   
4332773f-9505  lightgbm_4332773f-9505      17.582798       0.944541   
43327740-8500  lightgbm_43327740-8500      17.581506       0.944469   
43327741-b825  lightgbm_43327741-b825      17.581499       0.944468   
43327742-9534  lightgbm_43327742-9534      17.581569       0.944473   

              median_abs_value median_rel_value q25_abs_value q2

finished
finding best model for hour...
done

